In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('..\data\state_of_the_union.txt', encoding='utf8')
documents = loader.load()

In [3]:
print(documents[0].page_content[:500])

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six day


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [10]:
from langchain_community.embeddings import FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [11]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

In [13]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":3})

In [14]:
from langchain_groq import ChatGroq
llm = ChatGroq(model='openai/gpt-oss-120b', temperature=0.1)

In [15]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You are a helpful AI assistant. Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use the information to provide a concise and accurate answer.
Question: {question}
context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
print(rag_chain.invoke("Who is Madam Speaker and What is Her Speech About?"))

**Madam Speaker** – the title used for the presiding officer of the U.S. House of Representatives when that officer is a woman (the Speaker of the House at the time of the address).

**What her (the address’s) speech is about** – the President’s opening remarks to the joint session of Congress. In this portion he:

* Acknowledges the recent COVID‑19 pandemic and the fact that the nation is now gathering together again.  
* Calls for bipartisan unity – Democrats, Republicans and Independents – as “Americans” first.  
* Re‑affirms the nation’s commitment to the Constitution and to freedom.  
* Condemns Russia’s invasion of Ukraine, describing Vladimir Putin’s attempt to “shake the foundations of the free world” and praising the courage and determination of the Ukrainian people.  

So, “Madam Speaker” is the female Speaker of the House, and the speech she is hearing focuses on national recovery, bipartisan unity, and a strong stance against Russian aggression in Ukraine.
